# **Worksapce set up**

In [1]:
# Connecting to google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# Import libraries
%pylab inline
!pip install -q tensorflow-model-optimization

import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
import tempfile
import os
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

Populating the interactive namespace from numpy and matplotlib
     |████████████████████████████████| 237 kB 4.3 MB/s 


In [3]:
# Display python and library versions
!python --versions
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)
print('Keras ' + tf.keras.__version__)

unknown option --versions
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
Numpy 1.21.6
TensorFlow 2.8.2
Keras 2.8.0


# **Dataset Management**

In [4]:
# Loads the data and splits it into 60% training and 40% testing sets
df = loadtxt("drive/MyDrive/GE_practicum/diabetes.csv", delimiter=",", skiprows=1)

x_train = df[0:int(0.6*len(df)),0:8]
y_train = df[0:int(0.6*len(df)),8]

x_test = df[int(0.6*len(df)):,0:8]
y_test = df[int(0.6*len(df)):,8]

# **Quantization aware training**

In [5]:
# Loads the base model for ANN tests and gets baseline accuracy for result comparison
base_model = tf.keras.models.load_model('drive/MyDrive/GE_practicum/ANN_base')
base_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128)               1152      
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 11,521
Trainable params: 11,521
Non-trainable params: 0
_________________________________________________________________


# **Quantization and conversion to tflite**

In [6]:
# Conversion to tflite

converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
converter.target_spec.supported_ops = [
tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

open('ANN_quant.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpnounoqs3/assets


14768

In [7]:
# Check tflite model characteristics
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the FULLY_CONNECTED op takes
tensor #0 and tensor #5 and tensor #2 as input and produces tensor #9 as output.

Subgraph#0 main(T#0) -> [T#13]
  Op#0 FULLY_CONNECTED(T#0, T#5, T#2) -> [T#9]
  Op#1 FULLY_CONNECTED(T#9, T#6, T#1) -> [T#10]
  Op#2 FULLY_CONNECTED(T#10, T#7, T#4) -> [T#11]
  Op#3 FULLY_CONNECTED(T#11, T#8, T#3) -> [T#12]
  Op#4 LOGISTIC(T#12) -> [T#13]

Tensors of Subgraph#0
  T#0(serving_default_dense_8_input:0) shape_signature:[-1, 8], type:FLOAT32
  T#1(dense_9/bias) shape:[64], type:FLOAT32 RO 256 bytes
  T#2(dense_8/bias) shape:[128], type:FLOAT32 RO 512 bytes
  T#3(dense_11/bias) shape:[1], type:FLOAT32 RO 4 bytes
  T#4(dense_10/bias) shape:[32], type:FLOAT32 RO 128 bytes
  T#5(sequential_2/dense_8/MatMul) shape:[128, 8], type:INT8 RO 1024 bytes
  T#6(sequential_2/dense_9/MatMul) shape:[64, 128], type:INT

In [8]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
input_details = interpreter.get_input_details()

interpreter.allocate_tensors()
output_details = interpreter.get_output_details()

#Predictions from TFLite model
tfl_pred = []
tfl_pred_class = []
for i in range(len(x_test)):
    interpreter.set_tensor(input_details[0]["index"], x_test.astype('float32')[i:i+1,:])
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])
    tfl_pred.append(result)
    tfl_pred_class.append(int(result[0]*2))

right_pred = [y_test[i] == tfl_pred_class[i] for i in range(len(y_test))]
acc = sum(right_pred)/len(right_pred)
print(acc)

0.7337662337662337


# **Conversion to C array**

In [9]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [10]:
# Write TFLite model to a C source (or header) file
with open("ANN_quant" + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, "ANN_quant"))